#### Convert keypoints to geo-coord (visualization)

In [2]:
import os
import sys
sys.path.append(os.path.abspath('/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/src'))


In [ ]:
import pandas as pd
import json
from shapely.geometry import mapping, Point
from osgeo import gdal, osr
from utils.geo import translate_tiepoints_to_geo_coords, transform_to_epsg

sar_path = "/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/03_sar_train_01.tif"
opt_path = "/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/03_optical_train_01.tif"
tiepoints_path = "/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/03_tiepoints_train_01.csv"
output_geojson_path = "/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/03_tiepoints_train_01.geojson"

sar_ds = gdal.Open(sar_path)
opt_ds = gdal.Open(opt_path)
sar_gt = sar_ds.GetGeoTransform()
opt_gt = opt_ds.GetGeoTransform()

# === Get EPSG codes ===
sar_srs = osr.SpatialReference(wkt=sar_ds.GetProjection())
opt_srs = osr.SpatialReference(wkt=opt_ds.GetProjection())
sar_epsg = int(sar_srs.GetAttrValue('AUTHORITY', 1))
opt_epsg = int(opt_srs.GetAttrValue('AUTHORITY', 1))

# === Load tiepoints ===
df = pd.read_csv(tiepoints_path)

# === Convert image coords to geo coords ===
sar_y, sar_x = translate_tiepoints_to_geo_coords(df['sar_row'], df['sar_col'], sar_gt)
opt_y, opt_x = translate_tiepoints_to_geo_coords(df['optical_row'], df['optical_col'], opt_gt)

# === Reproject optical points to SAR CRS if needed ===
if opt_epsg != sar_epsg:
    opt_points = [Point(x, y) for x, y in zip(opt_x, opt_y)]
    opt_points_reprojected = transform_to_epsg(opt_points, opt_epsg, sar_epsg)
    opt_x = [p.x for p in opt_points_reprojected]
    opt_y = [p.y for p in opt_points_reprojected]

# === Prepare GeoJSON features ===
features = []
for i in range(len(sar_x)):
    sar_point = Point(sar_x[i], sar_y[i])
    opt_point = Point(opt_x[i], opt_y[i])
    features.append({
        "type": "Feature",
        "geometry": mapping(sar_point),
        "properties": {
            "type": "sar",
            "index": i
        }
    })
    features.append({
        "type": "Feature",
        "geometry": mapping(opt_point),
        "properties": {
            "type": "optical",
            "index": i
        }
    })

# === Create GeoJSON with CRS block ===
geojson = {
    "type": "FeatureCollection",
    "crs": {
        "type": "name",
        "properties": {
            "name": f"EPSG:{sar_epsg}"
        }
    },
    "features": features
}

# === Save to file ===
with open(output_geojson_path, 'w') as f:       
    json.dump(geojson, f, indent=2)

